Collecting Network Statistics in Pytorch
===

This is adapted from the workbook provided alongside the article "Implementing an Autoencoder in Pytorch" which can be found [here](https://medium.com/pytorch/implementing-an-autoencoder-in-pytorch-19baa22647d1). The purpose is to load generated trained models from the Autoencoder implementation and collect encoding statistics for analysis.


## Setup

We begin by importing our dependencies.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import math
import numpy

Set our seed and other configurations for reproducibility.

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

#if torch.cuda.is_available():
#    platform = "cuda"
#else:
#    platform = "cpu"
platform = "cpu"
print(platform)

We set the batch size, the number of training epochs, and the learning rate. Batch size has to be reasonably low as we can't fit a huge number of these images into VRAM on my laptop.

Image size can be set here as I'm automatically resizing the images in my extraction code.

In [ ]:
width = 256
height = 256

image_size = width * height

batch_size = 32

code_sides = [16]

model_path = "../../Data/OPTIMAM_NEW/model0.pt"

#image_count = 500
image_count = -1

## Dataset

ImageFolder is used to load the base distribution images. This version of the DataLoader setup is designed to not batch or shuffle the images as we load them sequentially

In [ ]:
from torchvision.datasets import ImageFolder

from torchvision.transforms import ToTensor,Grayscale
transform = torchvision.transforms.Compose([
     torchvision.transforms.Grayscale(),
     torchvision.transforms.Resize((height,width)),
     torchvision.transforms.ToTensor(),
     #torchvision.transforms.Normalize(mean=[0.5], std=[0.5])
    ])

root_dir = "../../Data/OPTIMAM_NEW/png_images"
train_dataset = torchvision.datasets.ImageFolder(root=root_dir, transform=transform)
if (image_count == -1):
    train_dataset_subset = train_dataset
    image_count = len(train_dataset)
    print("setting image count to " + str(image_count))
else:
    train_dataset_subset = torch.utils.data.Subset(train_dataset, numpy.random.choice(len(train_dataset), image_count, replace=False))

train_loader = torch.utils.data.DataLoader(
    train_dataset_subset, shuffle=False
)

### Autoencoder

We have to load the same encoding setup as in our autoencoder. (We'll set this up as an include in a bit.)

In [ ]:
class SplitAutoencoder(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder = nn.Sequential( 
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 128x128x32
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 64x64x64
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 32x32x128
            nn.Flatten(), # 131072x1
            nn.Linear(in_features=32*32*128,out_features=kwargs["code_size"]),
            nn.ReLU()
        )
        # result (encoding) is code_size x 1
        
        self.decoder = nn.Sequential(
            nn.Linear(in_features=kwargs["code_size"], out_features=32*32*128), #131072x1
            nn.Unflatten(1,(128,32,32)), # 32x32x128
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'), # 64x64x64
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'), # 128x128x32
            nn.Conv2d(in_channels=32, out_channels=1, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'), #256x256x1
        )
        
    def forward(self, features):
        code = self.encoder(features)
        out = self.decoder(code)
        return out

Before using our defined autoencoder class, we have the following things to do:
    1. We configure which device we want to run on.
    2. We instantiate our modules.
    3. We define our optimizer.
    4. We define our reconstruction loss.

In [ ]:
#  use gpu if available
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device(platform)

# reload the saved model

model = torch.load(model_path,map_location=device)
model.eval()

code_size = code_sides[0] * code_sides[0]

# mean-squared error loss
criterion = nn.MSELoss()
#criterion = nn.BCELoss()

losses = [None] * len(train_dataset_subset)
encodings = [None] * len(train_dataset_subset)

We run our autoencoder on the entire dataset and store the encodings

In [ ]:
with torch.no_grad():
    count = 0
    for batch_features, labels in train_loader:
        print("Feature Batch:" + str(count) + " Labels:" + str(labels[0]))
        # load it to the active device
        batch_features = batch_features.to(device)

        # compute reconstructions
        code = model.encoder(batch_features)
        outputs = model.decoder(code)
        
        code_reshaped = code.detach().numpy()[0]
        code_reshaped.reshape(code_size)

        encodings[count] = code_reshaped

        # compute training reconstruction loss
        error_criterion = criterion(outputs,batch_features)

        losses[count] = error_criterion.numpy()

        count = count + 1
    

Now we save the compiled statistics to an excel file.

In [ ]:
with torch.no_grad():
    np_losses = np.asarray(losses)

    np_compiled = np.concatenate((np_losses[:, np.newaxis], encodings), axis=1)

    np.savetxt('encodings.csv', encodings, delimiter=',',fmt='%10.5f',newline='\n')
    np.savetxt('losses.csv', np_losses, delimiter=',',fmt='%10.5f',newline='\n')
    np.savetxt('combined.csv', np_compiled, delimiter=',',fmt='%10.5f',newline='\n')

## Visualize Results

Let's try to reconstruct some test images using our trained autoencoder.

In [ ]:
with torch.no_grad():
    for i in range(len(code_sides)):
        for batch_features in test_loader:
            batch_features = batch_features[0]
            test_examples = batch_features.to(device)
            n_codes = models[i].encoder(test_examples)
            reconstruction = models[i](test_examples)
            break;
        test_example_sets[i] = test_examples
        code_sets[i] = n_codes
        reconstruction_sets[i] = reconstruction

            

In [ ]:
with torch.no_grad():
    for i in range(len(code_sides)):
        number = 5
        plt.figure(figsize=(25, 9))
        for index in range(number):
            # display original
            ax = plt.subplot(3, number, index + 1)
            test_examples = test_example_sets[i]
            copyback = test_examples[index].cpu()
            plt.imshow(copyback.numpy().reshape(height, width))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

            # display codes
            ax = plt.subplot(3, number, index + 1 + number)
            codes = code_sets[i]
            code_copyback = codes[index].cpu()
            plt.imshow(code_copyback.numpy().reshape(code_sides[i],code_sides[i]))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

            # display reconstruction
            ax = plt.subplot(3, number, index + 6 + number)
            reconstruction = reconstruction_sets[i]
            recon_copyback = reconstruction[index].cpu()
            plt.imshow(recon_copyback.numpy().reshape(height, width))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
        
        out_path = "output"+str(i)+".png" 
        plt.savefig(out_path)
        plt.show()